In [1]:
!pip install gym

In [2]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=b3508b278583a5326e183ee7d715640d23d8a2c2f73bd3003f1501ae07fbaa7f
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
import yfinance as yf
import pandas as pd
import ta  # For adding technical indicators

ModuleNotFoundError: No module named 'ta'

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
!pip install yfinance pandas ta

In [ ]:
# Load saved Bi-LSTM models
model_amzn = load_model('bi_lstm_amzn.keras')
model_nvda = load_model('bi_lstm_nvda.keras')
model_msft = load_model('bi_lstm_msft.keras')

C:\Users\djtur\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
class StockTradingEnvWithLSTM(gym.Env):
    def __init__(self, stock_data, lstm_models, initial_balance=10000):
        super(StockTradingEnvWithLSTM, self).__init__()

        # Store stock data and LSTM models
        self.stock_data = stock_data  # Historical stock prices
        self.lstm_models = lstm_models  # Dictionary containing Bi-LSTM models for each stock
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.holdings = {'AMZN': 0, 'NVDA': 0, 'MSFT': 0}
        self.current_step = 0

        # Action space: buy (0), sell (1), hold (2) for each of the 3 stocks
        self.action_space = spaces.MultiDiscrete([3, 3, 3])

        # Observation space: stock prices + LSTM predictions
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(stock_data.shape[1] + 3,), dtype=np.float32)

    def _get_observation(self):
        # Get the current stock prices for AMZN, NVDA, and MSFT
        stock_prices = self.stock_data.iloc[self.current_step].values

        # Prepare input for LSTM prediction (e.g., last 60 days of stock data)
        lstm_input_amzn = stock_prices.reshape(1, -1, 1)  # Reshape appropriately for each LSTM
        lstm_input_nvda = stock_prices.reshape(1, -1, 1)
        lstm_input_msft = stock_prices.reshape(1, -1, 1)

        # Get predictions from Bi-LSTM models for AMZN, NVDA, and MSFT
        predicted_amzn = self.lstm_models['AMZN'].predict(lstm_input_amzn).flatten()
        predicted_nvda = self.lstm_models['NVDA'].predict(lstm_input_nvda).flatten()
        predicted_msft = self.lstm_models['MSFT'].predict(lstm_input_msft).flatten()

        # Combine current stock prices with Bi-LSTM predictions into the observation space
        return np.concatenate([stock_prices, [predicted_amzn[0], predicted_nvda[0], predicted_msft[0]]])

    def step(self, action):
        # Execute the action (buy/sell/hold) for each stock
        self._take_action(action)

        # Move to the next time step
        self.current_step += 1

        # Get the next observation
        observation = self._get_observation()

        # Calculate the reward (change in portfolio value)
        reward = self._calculate_reward()

        # Check if the episode is done (when we reach the end of stock data)
        done = self.current_step >= len(self.stock_data) - 1

        return observation, reward, done, {}

    def _take_action(self, action):
        for idx, stock in enumerate(['AMZN', 'NVDA', 'MSFT']):
            current_price = self.stock_data.iloc[self.current_step][stock]
            if action[idx] == 0:  # Buy
                if self.balance >= current_price:
                    self.holdings[stock] += 1
                    self.balance -= current_price
            elif action[idx] == 1:  # Sell
                if self.holdings[stock] > 0:
                    self.holdings[stock] -= 1
                    self.balance += current_price
            # Action = 2 means Hold, so no action needed

    def _calculate_reward(self):
        # Calculate portfolio value as balance + value of holdings
        portfolio_value = self.balance
        for stock in ['AMZN', 'NVDA', 'MSFT']:
            portfolio_value += self.holdings[stock] * self.stock_data.iloc[self.current_step][stock]
        return portfolio_value - self.initial_balance

    def reset(self):
        # Reset the environment to the initial state
        self.balance = self.initial_balance
        self.holdings = {'AMZN': 0, 'NVDA': 0, 'MSFT': 0}
        self.current_step = 0
        return self._get_observation()

**Import Stock Data**

In [ ]:
# Define stock tickers and date range
tickers = ['AMZN', 'NVDA', 'MSFT']
start_date = '2020-01-01'
end_date = '2024-01-01'

# Fetch data for all tickers
stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Display first few rows of the data
print(stock_data.head())

[*********************100%%**********************]  3 of 3 completed

Ticker           AMZN                                                        \
Price            Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2020-01-02  93.750000  94.900497  93.207497  94.900497  94.900497  80580000   
2020-01-03  93.224998  94.309998  93.224998  93.748497  93.748497  75288000   
2020-01-06  93.000000  95.184502  93.000000  95.143997  95.143997  81236000   
2020-01-07  95.224998  95.694504  94.601997  95.343002  95.343002  80898000   
2020-01-08  94.902000  95.550003  94.321999  94.598503  94.598503  70160000   

Ticker            MSFT                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2020-01-02  158.779999  160.729996  158.330002  160.619995  153.938187   
2020-01-03  158.320007  159.949997  158.059998  158.619995  152.021393 

In [ ]:
# Function to add technical indicators for each stock
def add_indicators(df):
    df['SMA'] = ta.trend.sma_indicator(df['Close'], window=14)  # Simple Moving Average
    df['EMA'] = ta.trend.ema_indicator(df['Close'], window=14)  # Exponential Moving Average
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)  # Relative Strength Index
    df['MACD'] = ta.trend.macd(df['Close'])  # MACD
    bollinger = ta.volatility.BollingerBands(df['Close'])
    df['BB_upper'] = bollinger.bollinger_hband()
    df['BB_middle'] = bollinger.bollinger_mavg()
    df['BB_lower'] = bollinger.bollinger_lband()
    return df

# Process each stock separately and store in a list
processed_dataframes = []
for ticker in tickers:
    stock_df = stock_data[ticker].copy()  # Copy the stock data
    stock_df = add_indicators(stock_df)  # Add indicators
    stock_df['Ticker'] = ticker  # Add ticker column for identification
    processed_dataframes.append(stock_df)  # Append the processed DataFrame to the list

# Concatenate all processed DataFrames
final_stock_data = pd.concat(processed_dataframes)

# Display the processed data
print(final_stock_data.head())

Price            Open       High        Low      Close  Adj Close    Volume  \
Date                                                                          
2020-01-02  93.750000  94.900497  93.207497  94.900497  94.900497  80580000   
2020-01-03  93.224998  94.309998  93.224998  93.748497  93.748497  75288000   
2020-01-06  93.000000  95.184502  93.000000  95.143997  95.143997  81236000   
2020-01-07  95.224998  95.694504  94.601997  95.343002  95.343002  80898000   
2020-01-08  94.902000  95.550003  94.321999  94.598503  94.598503  70160000   

Price       SMA  EMA  RSI  MACD  BB_upper  BB_middle  BB_lower Ticker  
Date                                                                   
2020-01-02  NaN  NaN  NaN   NaN       NaN        NaN       NaN   AMZN  
2020-01-03  NaN  NaN  NaN   NaN       NaN        NaN       NaN   AMZN  
2020-01-06  NaN  NaN  NaN   NaN       NaN        NaN       NaN   AMZN  
2020-01-07  NaN  NaN  NaN   NaN       NaN        NaN       NaN   AMZN  
2020-01-08  Na

In [ ]:
# Save the final combined stock data with technical indicators to a CSV file
final_stock_data.to_csv('09_06_24_processed_historical_stock_data.csv')

**Initialize Models**

In [ ]:
pip uninstall torch torchvision torchaudio

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [ ]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return False if CPU mode

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\djtur\anaconda3\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
from stable_baselines3 import PPO

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\djtur\anaconda3\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
# Initialize the LSTM models in a dictionary for easier use in the environment
lstm_models = {
    'AMZN': model_amzn,
    'NVDA': model_nvda,
    'MSFT': model_msft
}

# Create the environment
env = StockTradingEnvWithLSTM(stock_data, lstm_models)

# Initialize the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the PPO model
model.learn(total_timesteps=5000)  # Adjust timesteps as needed

NameError: name 'PPO' is not defined

In [ ]:
# Test the trained PPO agent
obs = env.reset()
for _ in range(len(stock_data)):
    action, _states = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    if done:
        break

# Print the final portfolio value
final_portfolio_value = env.balance + sum([env.holdings[stock] * stock_data.iloc[env.current_step][stock] for stock in ['AMZN', 'NVDA', 'MSFT']])
print("Final portfolio value:", final_portfolio_value)

**Breakout for Alpaca API and implementation**

In [ ]:
pip install alpaca-trade-api

In [ ]:
import alpaca_trade_api as tradeapi

In [ ]:
# Alpaca API keys
API_KEY = 'your_api_key'
SECRET_KEY = 'your_secret_key'
BASE_URL = 'https://paper-api.alpaca.markets'

# Connect to Alpaca
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Check account details
account = api.get_account()
print(account)

In [ ]:
def execute_trade(action, stock):
    if action == 1:  # Buy
        api.submit_order(
            symbol=stock,
            qty=1,  # Buy one share
            side='buy',
            type='market',
            time_in_force='gtc'
        )
    elif action == 2:  # Sell
        api.submit_order(
            symbol=stock,
            qty=1,  # Sell one share
            side='sell',
            type='market',
            time_in_force='gtc'
        )

# Execute trades during market hours
obs = env.reset()
for _ in range(len(stock_data)):
    action, _states = model.predict(obs)
    for idx, stock in enumerate(['AMZN', 'NVDA', 'MSFT']):
        execute_trade(action[idx], stock)
    obs, reward, done, _ = env.step(action)
    if done:
        break